# 📌 TF-IDF (비슷한 마이룸 추천)

## ✏️ DB 연결

In [9]:
import os
import pymysql
import pandas as pd
import re

from dotenv import load_dotenv, find_dotenv


# 환경변수 불러오기
load_dotenv(find_dotenv())
HOST = os.environ["MYSQL_HOST"] # MySQL host
DB = os.environ["MYSQL_NAME"]   # MySQL name
USER = os.environ["MYSQL_USER"] # MySQL user
PASSWORD = os.environ["MYSQL_PASS"] # MySQL password
PORT = int(os.environ["MYSQL_PORT"]) # MySQL port


# DB 연결
db = pymysql.connect(host=HOST, port=PORT, user=USER, passwd=PASSWORD, db=DB, charset='utf8', autocommit=True, cursorclass=pymysql.cursors.DictCursor)
#  cursor생성
cursor = db.cursor()

In [13]:
# 한글, 숫자, 영문만 가져옴
def sub_special(s):
    # html 태그 제거
    tag_remover = re.compile('<.*?>')
    s = re.sub(tag_remover, '', s)
    return re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ]','',s)

# 리스트를 문자열로
def list2str(list):
    return "".join(list)

### ✏️ 로그인 한 유저의 포트폴리오 Summary 및 Tag 조회

In [17]:
# 로그인 한 유저의 번호
userSeq = 40

# 포트폴리오 summary 조회
port_sql = "SELECT DISTINCT(`p`.`summary`) AS `portfolios` \
  FROM `portfolio` AS `p` \
  INNER JOIN `arrange` AS `a` ON `p`.`port_seq` = `a`.`port_seq` \
  INNER JOIN `room` AS `r` ON `a`.`room_seq` = `r`.`room_seq` \
  INNER JOIN `user` AS `u` ON `r`.`user_seq` = `u`.`user_seq` \
  WHERE `u`.`user_seq` = " + str(userSeq) + ";"

cursor.execute(port_sql)
result = cursor.fetchall()

df_ports = pd.DataFrame(result)
ports = sub_special(list2str(list(df_ports["portfolios"])))

# Tag 조회
tag_sql = "SELECT DISTINCT(`t`.`name`) AS `tags` FROM `tag` AS `t` \
  INNER JOIN `arrange` AS `a` ON `t`.`port_seq` = `a`.`port_seq` \
  INNER JOIN `room` AS `r` ON `a`.`room_seq` = `r`.`room_seq` \
  INNER JOIN `user` AS `u` ON `r`.`user_seq` = `u`.`user_seq` \
  WHERE `u`.`user_seq` = " + str(userSeq) + ";"

cursor.execute(tag_sql)
result = cursor.fetchall()

df_tags = pd.DataFrame(result)
tags = sub_special(list2str(list(df_tags["tags"])))

feat_str = ports + tags
print(feat_str)

아이링크스스로 모든 것을 하고 싶어하는 모든 아이를 위해 우리 서비스는 만들어졌어요아이 혼자서 자신의 하루를 돌아보고 그 날의 기분을 기록해보아요매일 기록한 일상이 선생님과 부모님에게 전달되어 모두의 기억이 될 수 있게 해줄게요포트폴리오 내용nodejsexpress아이링크백엔드docker태그1태그2


### 형태소 분석

In [21]:
from konlpy.tag import Okt
okt = Okt()

file_path = "./stop_words.txt"
with open(file_path, encoding="utf8") as f:
    lines = f.readlines()
stop_words = [line.rstrip("\n") for line in lines]

In [24]:
# 형태소 분석
def morph_and_stopword(text):
    
    #형태소 분석
    words = okt.morphs(text, stem=True)
    
    # 형태소 분석 결과 담을 텍스트
    feat_list = []
    
    #불용어 처리
    for word in words:
        if word not in stop_words and len(word) > 1:
            feat_list.append(word)

    return list2str(set(feat_list))

In [27]:
text = morph_and_stopword(feat_str)

---

### 📌 모든 방의 포트폴리오 Summary, Tag

In [31]:
# 포트폴리오 summary 조회
port_sql = "SELECT DISTINCT(`p`.`summary`) AS `portfolios`, `r`.`room_seq` \
  FROM `portfolio` AS `p` \
  INNER JOIN `arrange` AS `a` ON `p`.`port_seq` = `a`.`port_seq` \
  INNER JOIN `room` AS `r` ON `a`.`room_seq` = `r`.`room_seq` \
  INNER JOIN `user` AS `u` ON `r`.`user_seq` = `u`.`user_seq`"

cursor.execute(port_sql)
result = cursor.fetchall()

df_ports = pd.DataFrame(result)
ports = sub_special(list2str(list(df_ports["portfolios"])))

# Tag 조회
tag_sql = "SELECT DISTINCT(`t`.`name`) AS `tags` FROM `tag` AS `t` \
  INNER JOIN `arrange` AS `a` ON `t`.`port_seq` = `a`.`port_seq` \
  INNER JOIN `room` AS `r` ON `a`.`room_seq` = `r`.`room_seq` \
  INNER JOIN `user` AS `u` ON `r`.`user_seq` = `u`.`user_seq` "

cursor.execute(tag_sql)
result = cursor.fetchall()

df_tags = pd.DataFrame(result)
tags = sub_special(list2str(list(df_tags["tags"])))

feat_str = ports + tags
print(feat_str)

아이링크스스로 모든 것을 하고 싶어하는 모든 아이를 위해 우리 서비스는 만들어졌어요아이 혼자서 자신의 하루를 돌아보고 그 날의 기분을 기록해보아요매일 기록한 일상이 선생님과 부모님에게 전달되어 모두의 기억이 될 수 있게 해줄게요포트폴리오 내용fsdfdsfdsfsfSFAFSFSGzzzzzzzzzzzzzzzzzzzzzzzznodejsexpress아이링크백엔드docker태그1태그2


### TEST TF-IDF

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

In [29]:
tf_idf = TfidfVectorizer()
tf_idf_matrix = tf_idf.fit_transform(feat_list)
print("TF-IDF 행렬의 크기(shape): ", tf_idf_matrix.shape)

cosine_sim = cosine_similarity(tf_idf_matrix, tf_idf_matrix)
print("코사인 유사도 연산 결과: ",cosine_sim.shape)

NameError: name 'feat_list' is not defined

[로그인 한 놈]  
- 모든 포트폴리오 목록 summary
- 모든 태그

[다른 사용자]  
- 방에 배치된 포트폴리오의 summary
- 해당 포트폴리오들의 태그  

유저-유저가 아닌 방-로그인 한 유저  

DataFrame  
[방 번호][포트폴리오내용][태그목록] 을
[방 번호][형태소분석 후의 feat str]로 만들어  
pickle로 저장 -> 데이터 로딩 시간 감소  



인덱스를 방 번호로 지정하고  
로그인한 유저의 텍스트랑  
방의 feat str의 유사도 계산  
유사도 순 정렬한 뒤 8개?  